<a href="https://colab.research.google.com/github/rickiepark/the-lm-book/blob/main/count_language_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<div style="display: flex; justify-content: center;">
    <div style="background-color: #f4f6f7; padding: 15px; width: 80%;">
        <table style="width: 100%">
            <tbody><tr>
                <td style="vertical-align: middle;">
                    <span style="font-size: 14px;">
                        A notebook for <a rel="noopener" href="https://www.thelmbook.com">The Hundred-Page Language Models Book</a> by Andriy Burkov<br><br>
                        Code repository: <a rel="noopener" href="https://github.com/aburkov/theLMbook">https://github.com/aburkov/theLMbook</a>
                    </span>
                </td>
                <td style="vertical-align: middle;">
                    <a href="https://www.thelmbook.com" target="_blank" rel="noopener">
                        <img src="https://thelmbook.com/img/book.png" width="80px" alt="The Hundred-Page Language Models Book">
                    </a>
                </td>
            </tr>
        </tbody></table>
    </div>
</div>

# 카운트 기반 언어 모델

## 유틸리티 함수와 클래스

다음 셀에서 필요 라이브러리를 임포트하고 유틸리티 함수와 모델 클래스를 정의합니다.

In [1]:
# 필수 라이브러리 임포트
import re         # 정규 표현식을 위한 라이브러리 (텍스트 토큰화)
import requests   # 말뭉치 다운로드를 위한 라이브러리
import gzip       # 다운로드한 말뭉치 압축 해제를 위한 라이브러리
import io         # 바이트 스트림 처리를 위한 라이브러리
import math       # 수학 연산을 위한 라이브러리 (로그, 지수)
import random     # 난수 생성을 위한 라이브러리
from collections import defaultdict  # 효율적인 딕셔너리 연산을 위한 라이브러리
import pickle, os # 모델 저장 및 로드를 위한 라이브러리

def set_seed(seed):
    """
    재현성을 위해 난수 시드를 설정합니다.
    Args:
        seed (int): 난수 생성기에 사용할 시드 값
    """
    random.seed(seed)

def download_corpus(url):
    """
    주어진 URL에서 gzip으로 압축된 말뭉치 파일을 다운로드하고 압축을 해제합니다.
    Args:
        url (str): gzip으로 압축된 말뭉치 파일의 URL
    Returns:
        str: 말뭉치의 디코딩된 텍스트 내용
    Raises:
        HTTPError: 다운로드 실패 시 발생하는 예외
    """
    print(f"{url}에서 말뭉치 다운로드 중...")
    response = requests.get(url)
    response.raise_for_status()  # 잘못된 HTTP 응답에 대해 예외를 발생시킴
    print("말뭉치 압축 해제 및 읽는 중...")
    with gzip.GzipFile(fileobj=io.BytesIO(response.content)) as f:
        corpus = f.read().decode('utf-8')
    print(f"말뭉치 크기: {len(corpus)} 문자")
    return corpus

class CountLanguageModel:
    """
    카운트 기반 확률 추정을 사용하는 n-그램 언어 모델을 구현합니다.
    n-그램까지 가변적인 문맥 길이를 지원합니다.
    """
    def __init__(self, n):
        """
        최대 n-그램 길이로 모델을 초기화합니다.
        Args:
            n (int): 사용할 n-그램의 최대 길이
        """
        self.n = n  # 최대 n-그램 길이
        self.ngram_counts = [{} for _ in range(n)]  # 각 n-그램 길이에 대한 딕셔너리 목록
        self.total_unigrams = 0  # 학습 데이터의 총 토큰 수

    def predict_next_token(self, context):
        """
        주어진 문맥에서 가장 가능성 있는 다음 토큰을 예측합니다.
        백오프 전략 사용: 가장 큰 n-그램부터 시도한 후 더 작은 n-그램으로 백오프합니다.
        Args:
            context (list): 예측을 위한 문맥을 제공하는 토큰 목록
        Returns:
            str: 가장 가능성 있는 다음 토큰, 예측을 할 수 없는 경우 None
        """
        for n in range(self.n, 1, -1):  # 가장 큰 n-그램으로 시작하여 더 작은 것으로 백오프
            if len(context) >= n - 1:
                context_n = tuple(context[-(n - 1):])  # 이 n-그램에 대한 관련 문맥 가져오기
                counts = self.ngram_counts[n - 1].get(context_n)
                if counts:
                    return max(counts.items(), key=lambda x: x[1])[0]  # 가장 빈번한 토큰 반환
        # 더 큰 문맥이 일치하지 않는 경우 유니그램으로 백오프
        unigram_counts = self.ngram_counts[0].get(())
        if unigram_counts:
            return max(unigram_counts.items(), key=lambda x: x[1])[0]
        return None

    def get_probability(self, token, context):
        for n in range(self.n, 1, -1):
            if len(context) >= n - 1:
                context_n = tuple(context[-(n - 1):])
                counts = self.ngram_counts[n - 1].get(context_n)
                if counts:
                    total = sum(counts.values())
                    count = counts.get(token, 0)
                    if count > 0:
                        return count / total
        unigram_counts = self.ngram_counts[0].get(())
        count = unigram_counts.get(token, 0)
        V = len(unigram_counts)
        return (count + 1) / (self.total_unigrams + V)

def train(model, tokens):
    """
    학습 데이터에서 n-그램을 계수하여 언어 모델을 학습시킵니다.
    Args:
        model (CountLanguageModel): 학습할 모델
        tokens (list): 학습 말뭉치의 토큰 목록
    """
    # 1부터 n까지 각 n-그램 크기에 대한 모델 학습
    for n in range(1, model.n + 1):
        counts = model.ngram_counts[n - 1]
        # 말뭉치 위에 크기 n의 창을 슬라이드
        for i in range(len(tokens) - n + 1):
            # 문맥(n-1 토큰)과 다음 토큰으로 분할
            context = tuple(tokens[i:i + n - 1])
            next_token = tokens[i + n - 1]
            # 필요한 경우 이 문맥에 대한 카운트 딕셔너리 초기화
            if context not in counts:
                counts[context] = defaultdict(int)
            # 이 문맥-토큰 쌍의 카운트 증가
            counts[context][next_token] = counts[context][next_token] + 1
    # 유니그램 확률 계산을 위한 총 토큰 수 저장
    model.total_unigrams = len(tokens)

def generate_text(model, context, num_tokens):
    """
    모델에서 반복적으로 샘플링하여 텍스트를 생성합니다.
    Args:
        model (CountLanguageModel): 학습된 언어 모델
        context (list): 초기 문맥 토큰
        num_tokens (int): 생성할 토큰 수
    Returns:
        str: 초기 문맥을 포함한 생성된 텍스트
    """
    # 제공된 문맥으로 시작
    generated = list(context)
    # 원하는 길이에 도달할 때까지 새 토큰 생성
    while len(generated) - len(context) < num_tokens:
        # 예측을 위한 문맥으로 마지막 n-1 토큰 사용
        next_token = model.predict_next_token(generated[-(model.n-1):])
        generated.append(next_token)
        # 충분한 토큰을 생성하고 마침표를 찾으면 중지
        # 이는 완전한 문장을 보장하는 데 도움이 됨
        if len(generated) - len(context) >= num_tokens and next_token == '.':
            break
    # 읽을 수 있는 텍스트를 만들기 위해 토큰을 공백으로 연결
    return ' '.join(generated)

def compute_perplexity(model, tokens, context_size):
    """
    주어진 토큰에 대한 모델의 혼잡도를 계산합니다.
    Args:
        model (CountLanguageModel): 학습된 언어 모델
        tokens (list): 평가할 토큰 목록
        context_size (int): 고려할 최대 문맥 크기
    Returns:
        float: 혼잡도 점수 (낮을수록 좋음)
    """
    # 빈 토큰 목록 처리
    if not tokens:
        return float('inf')
    # 로그 우도 누적 변수 초기화
    total_log_likelihood = 0
    num_tokens = len(tokens)
    # 각 토큰의 문맥에 주어진 확률 계산
    for i in range(num_tokens):
        # 적절한 문맥 윈도 가져오기, 시퀀스 시작 처리
        context_start = max(0, i - context_size)
        context = tuple(tokens[context_start:i])
        token = tokens[i]
        # 주어진 문맥에 대한 토큰의 확률 가져오기
        probability = model.get_probability(token, context)
        # 로그 확률 누적 (수치적 안정성을 위해 로그 사용)
        total_log_likelihood += math.log(probability)
    # 평균 로그 우도 계산
    average_log_likelihood = total_log_likelihood / num_tokens
    # 혼잡도로 변환: exp(-평균 로그 우도)
    # 낮은 혼잡도는 더 나은 모델 성능을 나타냄
    perplexity = math.exp(-average_log_likelihood)
    return perplexity

def tokenize(text):
    """
    텍스트를 단어와 마침표로 토큰화합니다.
    Args:
        text (str): 토큰화할 입력 텍스트
    Returns:
        list: 단어나 마침표와 일치하는 소문자 토큰 목록
    """
    return re.findall(r"\b[a-zA-Z0-9]+\b|[.]", text.lower())

def download_and_prepare_data(data_url):
    """
    훈련 및 테스트 데이터를 다운로드하고 준비합니다.
    Args:
        data_url (str): 다운로드할 말뭉치의 URL
    Returns:
        tuple: (훈련_토큰, 테스트_토큰) 90/10으로 분할
    """
    # 말뭉치 다운로드 및 추출
    corpus = download_corpus(data_url)
    # 텍스트를 토큰으로 변환
    tokens = tokenize(corpus)
    # 훈련(90%) 및 테스트(10%) 세트로 분할
    split_index = int(len(tokens) * 0.9)
    train_corpus = tokens[:split_index]
    test_corpus = tokens[split_index:]
    return train_corpus, test_corpus

def save_model(model, model_name):
    """
    훈련된 언어 모델을 디스크에 저장합니다.
    Args:
        model (CountLanguageModel): 저장할 훈련 모델
        model_name (str): 저장된 모델 파일에 사용할 이름
    Returns:
        str: 저장된 모델 파일의 경로
    Raises:
        IOError: 디스크 쓰기 오류가 있는 경우
    """
    # 모델 디렉토리가 없는 경우 생성
    os.makedirs('models', exist_ok=True)
    # 파일 경로 구성
    model_path = os.path.join('models', f'{model_name}.pkl')
    try:
        print(f"{model_path}에 모델 저장 중...")
        with open(model_path, 'wb') as f:
            pickle.dump({
                'n': model.n,
                'ngram_counts': model.ngram_counts,
                'total_unigrams': model.total_unigrams
            }, f)
        print("모델이 성공적으로 저장되었습니다.")
        return model_path
    except IOError as e:
        print(f"모델 저장 오류: {e}")
        raise

def load_model(model_name):
    """
    디스크에서 훈련된 언어 모델을 로드합니다.
    Args:
        model_name (str): 로드할 모델의 이름
    Returns:
        CountLanguageModel: 로드된 모델 인스턴스
    Raises:
        FileNotFoundError: 모델 파일이 존재하지 않는 경우
        IOError: 파일 읽기 오류가 있는 경우
    """
    model_path = os.path.join('models', f'{model_name}.pkl')
    try:
        print(f"{model_path}에서 모델 로드 중...")
        with open(model_path, 'rb') as f:
            model_data = pickle.load(f)
        # 새 모델 인스턴스 생성
        model = CountLanguageModel(model_data['n'])
        # 모델 상태 복원
        model.ngram_counts = model_data['ngram_counts']
        model.total_unigrams = model_data['total_unigrams']
        print("모델이 성공적으로 로드되었습니다.")
        return model
    except FileNotFoundError:
        print(f"모델 파일을 찾을 수 없음: {model_path}")
        raise
    except IOError as e:
        print(f"모델 로드 오류: {e}")
        raise

def get_hyperparameters():
    """
    모델 하이퍼파라미터를 반환합니다.
    Returns:
        int: 모델에서 사용할 n-그램 크기
    """
    n = 5
    return n

## 모델 훈련하기

다음 셀에서 데이터를 로드하고, 모델을 훈련 및 저장합니다.

In [2]:
# 재현성을 위해 랜덤 시드를 설정합니다.
set_seed(42)
n = get_hyperparameters()
model_name = "count_model"

# Brown 말뭉치를 다운로드하여 준비합니다.
data_url = "https://www.thelmbook.com/data/brown"
train_corpus, test_corpus = download_and_prepare_data(data_url)

# 모델을 훈련하고 성능을 평가합니다.
print("\n모델 훈련 중...")
model = CountLanguageModel(n)
train(model, train_corpus)
print("\n모델 훈련 완료.")

perplexity = compute_perplexity(model, test_corpus, n)
print(f"\n테스트 말뭉치에 대한 혼잡도: {perplexity:.2f}")

save_model(model, model_name)

https://www.thelmbook.com/data/brown에서 말뭉치 다운로드 중...
말뭉치 압축 해제 및 읽는 중...
말뭉치 크기: 6185606 문자

모델 훈련 중...

모델 훈련 완료.

테스트 말뭉치에 대한 혼잡도: 299.06
models/count_model.pkl에 모델 저장 중...
모델이 성공적으로 저장되었습니다.


'models/count_model.pkl'

## 모델 테스트하기

아래 셀에서 훈련된 모델을 로드하여 텍스트를 생성합니다.

In [3]:
model = load_model(model_name)

# 샘플 문맥으로 모델을 테스트합니다.
contexts = [
    "i will build a",
    "the best place to",
    "she was riding a"
]

# 각 문맥에 대해 완성을 생성합니다.
for context in contexts:
    tokens = tokenize(context)
    next_token = model.predict_next_token(tokens)
    print(f"\n문맥: {context}")
    print(f"다음 토큰: {next_token}")
    print(f"생성된 텍스트: {generate_text(model, tokens, 10)}")

models/count_model.pkl에서 모델 로드 중...
모델이 성공적으로 로드되었습니다.

문맥: i will build a
다음 토큰: wall
생성된 텍스트: i will build a wall to keep the people in and added so long

문맥: the best place to
다음 토큰: live
생성된 텍스트: the best place to live in 30 per cent to get happiness for yourself

문맥: she was riding a
다음 토큰: horse
생성된 텍스트: she was riding a horse and showing a dog are very similar your aids
